## Initialising Enviroment Variables

In [3]:
import os
import openai
import sys
sys.path.append("../..")

from dotenv import load_dotenv, find_dotenv
_= load_dotenv(find_dotenv())

openai.api_key=os.environ["OPENAI_API_KEY"]

In [4]:
from langchain.document_loaders import PyPDFLoader

In [5]:
loader = PyPDFLoader("./docs/Saloni_Kumari_Resume.pdf")
pages=loader.load()

In [6]:
page=pages[0]
print(page.page_content[:100])

PROFESSIONAL SUMMARY
An enthus iastic full-stack web developer , a lifelong
learner skilled in both 


In [ ]:
# Splitting the document

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [7]:
chunk_size = 20
chunk_overlap = 4

In [10]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap =chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap =chunk_overlap
)


In [11]:
text_splitter= CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

In [12]:
docs = text_splitter.split_documents(pages)

In [13]:
len(docs)

3

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [15]:
splits=text_splitter.split_documents(docs)

In [16]:
len(splits)

3

In [17]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [18]:
sen1="I love dogs"
sen2="I like cats"
sen3="wheather is rainy"

In [19]:
from langchain.vectorstores import Chroma

In [20]:
persist_directory ="docs/chroma"

In [21]:
!rm -rf ./docs/chroma # remove old database file if any

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [22]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [23]:
print(vectordb._collection.count())

6


In [24]:
question = "full"

In [25]:
docs= vectordb.similarity_search(question,k=3)

In [26]:
docs[0].page_content

"PROFESSIONAL SUMMARY\nAn enthus iastic full-stack web developer , a lifelong\nlearner skilled in both front-end and back-end\ndevelopment, with a high adaptability to learn new\ntechnologies quickly .The goal is to leverage\nknowledge and build world-class web applications\nwhile facilitating the organization in achie ving\nfunctional goals.\nEDUCATION\nFull Stack Web Development | Full Time, \nMasai School, Bangalor e\nOct\xa02022 – Jul\xa02023  | Bangalor e, Karnataka\nBachelor 's In Computer Application, \nL.S.College, Muzaffarpur\nSep\xa02017 – Apr\xa02020  | Muzaffarpur , Bihar\nSKILLS\nHTML |CSS |JavaScript |Node.Js\nExpr ess.Js |MongoDB |MySql |TypeScript\nReact |AWS |SQL |Swagger |Bootstr ap\nSystem Design |Redis |Python\nSOFT SKILLS\nLeadership\nProblem-Solving\nTime Management\nTeam Collabor ation\nINTERESTS\nYoga\nCodingPROJECTS\nCLONE OF TRIPVILL A\nBook Top Holida y Homes - Villas, Apar tments &  \nHomesta ys\nGITHUB\nTECH ST ACK : HTML | CSS | Ja va Script\nFEATURES :\n•

# Retreival

In [39]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [40]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)

In [42]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [46]:
question = "on which topic it is talking"

In [47]:
result = qa_chain({"query": question})

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


In [48]:
result["result"]

'The topic being discussed is a coding project that is a clone of TripVilla, a platform for booking holiday homes, villas, apartments, and homestays. The project involves HTML, CSS, and JavaScript and includes features such as sign-in, sign-up, and home page and hotel page functionalities.'

In [49]:
from langchain.memory import ConversationBufferMemory

In [63]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [64]:
from langchain.chains import ConversationalRetrievalChain 

In [65]:
retriever = vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory

)

In [66]:
question="what do you know about saloni" 


In [67]:
result =  qa({"question" : question})


Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


In [68]:
result["answer"]

"Saloni Kumari is a full-stack web developer with experience in both front-end and back-end development. She has a strong ability to quickly learn new technologies and is skilled in HTML, CSS, JavaScript, Node.js, Express.js, MongoDB, and Redis. Saloni has completed a Full Stack Web Development program at Masai School in Bangalore. She also holds a Bachelor's degree in Computer Application from L.S. College in Muzaffarpur, Bihar. Saloni has worked on collaborative projects with teams of 5 members, completing them in a span of 5 days. She has also worked on individual projects. If you have any specific questions about Saloni, feel free to ask."